In [392]:
# Imports
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import optuna
import joblib

import torch

from darts import TimeSeries
from darts.models import  RandomForest, LinearRegressionModel, LightGBMModel, \
                        CatBoostModel,  BlockRNNModel, NBEATSModel, NHiTSModel, \
                        TCNModel, TFTModel
from darts.dataprocessing.transformers import Scaler

from darts.utils.likelihood_models import GaussianLikelihood

import warnings
warnings.filterwarnings('ignore')

# Configuration

In [393]:
prj_path = '../'
data_path = prj_path + "data/new_data/DH/squeezed/"
prj_path_opt= prj_path + "optimize_hyperparam/opt_results/"
output_process = prj_path + "data/new_data/DH/processed_data/"
output_featureselection = prj_path + "data/new_data/DH/feature_selection/"

In [394]:
all_cities = [
        'An Giang', 'BR Vũng Tàu', 'Bình Phước', 'Bình Thuận', 'Bình Định',
        'Bạc Liêu', 'Bắc Kạn', 'Bắc Giang', 'Cao Bằng', 'Cà Mau',
        'Cần Thơ', 'Gia Lai', 'Hà Giang', 'Hà Nội', 'Hà Tĩnh',
        'Hòa Bình','Hưng Yên', 'Hải Dương', 'Hải Phòng', 'Khánh Hòa', 'Kiên Giang',
        'Kon Tum', 'Lai Châu', 'Long An', 'Lào Cai', 'Lâm Đồng',
        'Lạng Sơn','Nam Định', 'Nghệ An', 'Ninh Bình', 'Ninh Thuận',
        'Phú Thọ', 'Phú Yên', 'Quảng Bình', 'Quảng Nam', 'Quảng Ngãi',
        'Quảng Ninh', 'Quảng Trị', 'Sóc Trăng', 'Sơn La', 'TT Huế',
        'Thanh Hóa', 'Thái Bình', 'Thái Nguyên', 'Tiền Giang', 'Trà Vinh',
        'Tuyên Quang', 'Tây Ninh', 'Vĩnh Phúc', 'Yên Bái', 'Điện Biên',
        'Đà Nẵng', 'Đắk Nông', 'Đắk Lắk', 'Đồng Tháp'
]
cities = ['An Giang', 'BR Vũng Tàu', 'Bình Phước', 'Bình Thuận', 'Bình Định']

In [395]:
# Set hyperparameters as args using the Configuration class
class Configuration():
    def __init__(self):
        # lấy bộ test dài 36 tháng = 3 năm
        self.test_size = 36
        # là nhìn vào dữ liệu trước 3 tháng và dự phóng        
        self.look_back = 3
        # dự phóng n-step trong 6 tháng
        self.n_predicted_period_months = 6
        self.n_features = 3
        self.seed = 42
        # mỗi phần tử x trong tập suppervise có độ lớn là 16 = 16 tháng
        self.batch_size = 16
        self.device = torch.device("cuda")
        self.epochs = 300
        #others
        self.labels = "Dengue_fever_rates"
        # Input param for Optimize Run
        self.ntry = 1
        self.njob = 1

args = Configuration()

# Seeding

In [396]:
def seed_everything(seed: int):
    import random
    import numpy as np
    import torch
    
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

seed_everything(args.seed)

# Supporting functions

In [ ]:
import requests

def send_to_telegram(message):

    apiToken = '5908735099:AAGVSLrW62aXPBP-GrMvxoVgMsuJxXJpP1Q'
    chatID = '@ptn_announcement'
    apiURL = f'https://api.telegram.org/bot{apiToken}/sendMessage'

    try:
        response = requests.post(apiURL, json={'chat_id': chatID, 'text': message})
        print(response.text)
    except Exception as e:
        message_error = "Bị lỗi rùi: "+str(e)
        response = requests.post(apiURL, json={'chat_id': chatID, 'text': message_error})
        print(e)

In [397]:
def get_dict_all_city_data():
  cities_data = {}  
  for city in cities:
    city_result = pd.read_excel(prj_path+'data/new_data/DH/squeezed/squeezed_'+city+'.xlsx')  
    """Get all data from all city in 1997 - 2016""" 
    city_result = city_result.loc[city_result['year_month'] < '2017-1-1'] 
    cities_data[city] = city_result
  return cities_data

In [398]:
# Define data (pre-)processing functions
# modification
def get_city_data(city_name, dict_full_data):
    """Returns Diarrhoea rate and climate data""" 
    city_data = dict_full_data[city_name].drop(columns=['Diarrhoea_cases','Diarrhoea_rates', 'province',
                                                        'Influenza_rates','Influenza_cases',
                                                        'Dengue_fever_cases', 'year', 'month'], 
                                                                  axis=1, 
                                                                  inplace=False)    
    return city_data

def convert_to_stationary(city_data):
    """Subtracts previous value for all cols except disease rates"""
    for col_name in city_data.columns:
        if col_name != 'Diarrhoea_rates':
            try:
                city_data[col_name] = city_data[col_name] - city_data[col_name].shift()
            except:
                print(col_name)
    return city_data

def impute_missing_value(city_data):
    """
    Imputes 0 for first 12 months, 
    last year's value for months 12-24, 
    and minimum value of last two years for months 25+
    """
    for col in city_data.columns:
        for index in range(len(city_data[col])):
            if np.isnan(city_data[col].iloc[index]):
                if index < 12:
                    city_data[col].iloc[index] = 0
                elif index >= 12 and index <= 24:
                    city_data[col].iloc[index] = city_data[col].iloc[index - 12]
                else:
                    city_data[col].iloc[index] = min(city_data[col].iloc[index - 12], city_data[col].iloc[index - 24])
    return city_data

In [399]:
def clean_full_data(dict_full_data):
    climate_and_disease_feats = ['Total_Evaporation',
       'Total_Rainfall', 'Max_Daily_Rainfall', 'n_raining_days',
       'Average_temperature', 'Max_Average_Temperature',
       'Min_Average_Temperature', 'Max_Absolute_Temperature',
       'Min_Absolute_Temperature', 'Average_Humidity', 'Min_Humidity',
       'n_hours_sunshine', 'Dengue_fever_rates']
    for city in cities:
        city_data = get_city_data(city_name=city,dict_full_data = dict_full_data)
        city_data_features = city_data[climate_and_disease_feats]
        city_data_features = impute_missing_value(city_data_features)
        city_data_features = convert_to_stationary(city_data_features)
        city_data_features.dropna(inplace=True)
        city_data_features.loc[:, "year_month"] = city_data["year_month"]
        dict_full_data[city] = city_data_features
    return dict_full_data


In [400]:
def split_data(data, look_back, n_nextstep = args.n_predicted_period_months):
    """Splits data into train and test sets based on args (Configuration class)"""
    train = data[: -args.test_size]    
    test = data[-args.test_size - look_back-(n_nextstep - 1): ]
    return train, test

In [401]:
def to_supervised(data,  d_out, d_in, features_list=[]):
    """
    Frames time-series as supervised learning dataset.
    
    Args:
      d_in: lookback window
      d_out: number of predicted months
      features_list: list of all features **where last col is the disease incidence**

    Returns:
      Numpy arrays of disease incidence (y) and other predictors (X)
    """
    X, y = list(), list()
    for index, _ in enumerate(data):
        in_end = index + d_in
        out_end = in_end + d_out
        if out_end <= len(data):
            if len(features_list) == 0 :
                X.append(data[index: in_end, :])
            else:
                X.append(data[index: in_end, features_list])
            y.append(data[out_end-1: out_end, -1])
    return np.array(X), np.array(y)

In [402]:
def train_and_evaluate(df_train, df_eval, model, feature_list , labels, scaler):
  """
  $df: pandas.DataFrame object containing data for training and testing model:
  $model: darts model object
  $feature_list: Names of the features used as model input
  $label: the value the model will be trained to predict
  $scaler: scaler object. Note: the scaler will be fitted on training data and applied to test data
  $lags: how much to look back into the past to output prediction
  $split_index: the point at which to divide train and test_data

  """

  x_train = TimeSeries.from_dataframe(df_train, "year_month", feature_list)
  y_train = TimeSeries.from_dataframe(df_train, "year_month", labels)

  x_test = TimeSeries.from_dataframe(df_eval, "year_month", feature_list)
  y_test = TimeSeries.from_dataframe(df_eval, "year_month", labels)

  model.fit(y_train, past_covariates = x_train)

  prediction = model.predict(len(y_test)-args.look_back, past_covariates = x_test, num_samples=1)

  y_true = scaler.inverse_transform(df_eval.iloc[:,:-1])[:,[-1]].reshape(len(df_eval))[args.look_back:]

  df_eval[labels][args.look_back:] = np.array(prediction._xa).squeeze()
  y_pred = scaler.inverse_transform(df_eval.iloc[args.look_back:,:-1])[:,[-1]].reshape(len(prediction))

  # df_compare_test_predict = pd.DataFrame({'y_true':y_true, 'y_pred':y_pred})
  # df_compare_test_predict.plot()
  # plt.legend()
  # plt.show()

  mse = mean_squared_error(y_true, y_pred)
  mae = mean_absolute_error(y_true, y_pred)
  rmse = mse**0.5
  mape = mean_absolute_percentage_error(y_true, y_pred)
  print(f"mean_squared_error: {mse:.4f}")
  print(f"rmse: {rmse}")
  print(f"mape: {mape}")
  return model, y_true, y_pred, mse, mae, rmse, mape

In [403]:
def output_prediction_for_location(df_train, df_eval, model, location, feature_list, 
                                                labels, scaler):
    """train and generate prediction for a province
    df: DataFrame object containing features and label(s) for training model
    localtion: location_name
    feature_list: list of features used as model input,  must be among the column names of df
    labels: the values model will be trained to predict
    scaler: sklearn scaler object
    lags: how long into the past to look back when making prediction
    split_index: the point at which to divide data into the train and test subsets.
    """
    model, y_true, prediction_inverse, mse, mae, rmse, mape = train_and_evaluate(df_train, df_eval, model, feature_list, labels, scaler)
    df_prediction = pd.DataFrame({"Date": df_eval["year_month"][-len(prediction_inverse):],
                                  "Observed": y_true[-len(prediction_inverse):],
                                  "1-month": prediction_inverse})
    
    df_prediction["City"] = location
    df_prediction["RMSE_1-month"] = rmse
    df_prediction["MAE_1-month"] = mae
    df_prediction["MAPE_1-month"] = mape

    return mae

In [404]:
def getDataWithSelectedFeature(city, next_predicted_month):
  selected_feature = []
  df = pd.read_csv(output_featureselection+str(next_predicted_month)+"step_feature_selection_3_most.csv", encoding = 'unicode_escape')
  for row in range(len(df)):
    if (df["City"][row] == city):
      selected_feature.append(df["1st_Feature"][row])
      selected_feature.append(df["2nd_Feature"][row])
      selected_feature.append(df["3rd_Feature"][row])
  return selected_feature

# Objective and Suggest Hyperparams of Darts Models


In [405]:
def objective(model_name, trial, city, nstep):   
    specific_data = pd.read_csv(output_process+city+'_train_preprocessed.csv', parse_dates=True, index_col= None, encoding = 'unicode_escape')
    scaler = joblib.load(output_process+city+'_train_scalerMinMaxNorm.save') #ok

    df_train, df_valid = split_data(specific_data, args.look_back,nstep)
    selected_features = getDataWithSelectedFeature(city, nstep)

    lags_by_nstep = args.look_back + nstep - 1
    lags_past_covariates_by_nstep = [-lags_by_nstep+2,-lags_by_nstep+1,-lags_by_nstep] #Mảng này chứa ba giá trị tương ứng cho args.lookback 3

    pl_trainer_kwargs = {
              "accelerator": "gpu",
              "devices": -1,
              "auto_select_gpus": True,
          }


    if model_name == "RandomForest":
      random_state = trial.suggest_int('random_state', 0, 42)
      n_estimators = trial.suggest_int('n_estimators', 50, 200)
      max_depth = trial.suggest_int('max_depth', 1, 15)
      # Create the RandomForest model
      model = RandomForest(
                    lags = lags_by_nstep,
                    lags_past_covariates = lags_past_covariates_by_nstep,
                    output_chunk_length = 1,
                    n_estimators = n_estimators,
                    max_depth = max_depth,
                    random_state=random_state)
    elif model_name == 'TFTModel':
      # Define the hyperparameters to optimize
      random_state = trial.suggest_int('random_state', 0, 42)
      dropout = trial.suggest_uniform('dropout', 0.01, 0.8)
      n_epochs = trial.suggest_int('n_epochs', 50, 200)

      # Create the TFTModel model
      model = TFTModel(
                    input_chunk_length = lags_by_nstep,
                    output_chunk_length = 1,
                    add_relative_index = True,
                    dropout = dropout,
                    n_epochs = n_epochs ,
                    random_state=random_state)
    elif model_name == 'NHiTSModel':
      #suggest hyperparams
      random_state = trial.suggest_int('random_state', 0, 42)
      dropout = trial.suggest_uniform('dropout', 0.01, 0.80)
      n_epochs = trial.suggest_int('n_epochs', 100, 500, step=10)
      MaxPool1d = trial.suggest_categorical('MaxPool1d', [True, False])

      model = NHiTSModel(
                          input_chunk_length = lags_by_nstep,
                          output_chunk_length = 1,
                          MaxPool1d = MaxPool1d,
                          dropout = dropout,
                          n_epochs = n_epochs ,
                          pl_trainer_kwargs = pl_trainer_kwargs,
                          random_state=random_state)
    elif model_name == 'LinearRegressionModel':
      random_state = trial.suggest_int('random_state', 0, 43)
      
      # Create the  model
      model = LinearRegressionModel(
                      lags = lags_by_nstep,
                      lags_past_covariates = lags_past_covariates_by_nstep,
                      output_chunk_length = 1,
                      random_state=random_state)
    elif model_name == "BlockRNNModel":
      #suggest hyperparams
      random_state = trial.suggest_int('random_state', 0, 1000)
      n_rnn_layers = trial.suggest_int('n_rnn_layers', 1, 3)
      dropout = trial.suggest_uniform('dropout', 0.1, 0.5)
      hidden_dim = trial.suggest_int('n_rnn_layers', 5, 20)
      n_epochs = trial.suggest_int('n_epochs', 50, 200)

      model = BlockRNNModel(
                          input_chunk_length = lags_by_nstep,
                          output_chunk_length = 1,
                          hidden_dim = hidden_dim,
                          n_rnn_layers = n_rnn_layers,
                          dropout = dropout,
                          n_epochs = n_epochs,
                          pl_trainer_kwargs = pl_trainer_kwargs,
                          random_state=random_state)
      
    elif model_name == "CatBoostModel":
      #suggest hyperparams
      learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
      n_estimators = trial.suggest_int('n_estimators', 50, 200)
      max_depth = trial.suggest_int('max_depth', 1, 15)
      random_state = trial.suggest_int('random_state', 0, 1000)
      likelihood = trial.suggest_categorical('likelihood', ['quantile'])
      quantiles =  trial.suggest_categorical('quantiles', [None, [0.1, 0.5, 0.9]])
      bagging_temperature = trial.suggest_float('bagging_temperature', 0.01, 100.0)
      border_count = trial.suggest_int('border_count', 1, 255)
      l2_leaf_reg = trial.suggest_float('l2_leaf_reg', 0.1, 10)
      random_strength = trial.suggest_float('random_strength', 0.1, 10)
      model = CatBoostModel(
                            lags=lags_by_nstep,
                            lags_past_covariates=lags_past_covariates_by_nstep, 
                            learning_rate=learning_rate,
                            n_estimators=n_estimators,
                            max_depth=max_depth, 
                            output_chunk_length = 1,
                            likelihood = likelihood,
                            quantiles = quantiles,
                            bagging_temperature = bagging_temperature,
                            border_count = border_count,
                            l2_leaf_reg = l2_leaf_reg,
                            random_strength = random_strength,
                            random_state=random_state)
    
    elif model_name == "NBEATSModel":
      random_state = trial.suggest_int('random_state', 0, 42)
      dropout = trial.suggest_uniform('dropout', 0.01, 0.80)
      n_epochs = trial.suggest_int('n_epochs', 50, 200)

    
      model = NBEATSModel(
                            input_chunk_length = lags_by_nstep,
                            output_chunk_length = 1,
                            dropout = dropout,
                            n_epochs = n_epochs ,
                            pl_trainer_kwargs = pl_trainer_kwargs,
                            random_state=random_state)
  
    elif model_name == "TCNModel":
      params = {
        'kernel_size': trial.suggest_int("kernel_size", 2, lags_by_nstep),
        'num_filters': trial.suggest_int("num_filters", 1, 5),
        'weight_norm': trial.suggest_categorical("weight_norm", [False, True]),
        'dilation_base': trial.suggest_int("dilation_base", 2, 4),
        'dropout': trial.suggest_float("dropout", 0.0, 0.4),
        'learning_rate': trial.suggest_float("learning_rate", 5e-5, 1e-3, log=True),
        'include_year': trial.suggest_categorical("year", [False, True]),
        'n_epochs': trial.suggest_int("n_epochs", 100, 300),
      }

      # select input and output chunk lengths

      params['input_chunk_length'] = lags_by_nstep
      params['output_chunk_length'] = 1  
   

      # optionally also add the (scaled) year value as a past covariate
      if params['include_year']:
          encoders = {"datetime_attribute": {"past": ["year"]},
                      "transformer": Scaler()}
      else:
          encoders = None
      params['encoders'] = encoders


      param = params
      model = TCNModel(
          input_chunk_length=param['input_chunk_length'],
          output_chunk_length=param['output_chunk_length'],
          batch_size=16,
          n_epochs=param['n_epochs'],
          nr_epochs_val_period=1,
          kernel_size=param['kernel_size'],
          num_filters=param['num_filters'],
          weight_norm=param['weight_norm'],
          dilation_base=param['dilation_base'],
          dropout=param['dropout'],
          optimizer_kwargs={"lr": param['learning_rate']},
          add_encoders=param['encoders'],
          likelihood=GaussianLikelihood(),
          pl_trainer_kwargs=pl_trainer_kwargs,
          model_name="tcn_model",
          force_reset=True,
          save_checkpoints=True,
      )
    elif model_name == "LightGBMModel":
      params = {
        "lags": lags_by_nstep,
        "lags_past_covariates": lags_past_covariates_by_nstep,
        "random_state": trial.suggest_int("random_state", 0, 999),
        "multi_models": trial.suggest_categorical("multi_models", [True, False]),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'verbose': -1,
        'likelihood' : trial.suggest_categorical("likelihood", ["quantile"])
      }

      param = params
      model = LightGBMModel(
          lags = param['lags'],
          lags_past_covariates = param['lags_past_covariates'],
          output_chunk_length = 1,
          random_state = param['random_state'],
          multi_models = param['multi_models'],
          likelihood = param['likelihood'],
          num_leaves = param['num_leaves'],
          learning_rate = param['learning_rate'],
          feature_fraction = param['feature_fraction'],
          bagging_fraction = param['bagging_fraction'],
          min_child_samples = param['min_child_samples'],
          lambda_l1 = param['lambda_l1'],
          verbose = param['verbose']
      )
    
    mae_error = output_prediction_for_location(df_train, df_valid, model, location=city, feature_list=selected_features,
                                                labels=args.labels, scaler=scaler)

    return mae_error

# Main run optimize and save

In [408]:
#########################
# Main Cell for optimize
#########################

model_name_list = [
     "RandomForest",
     "LinearRegressionModel",
     "LightGBMModel",
     "CatBoostModel",
     "BlockRNNModel",
     "NBEATSModel",
     "NHiTSModel",
     "TCNModel",
     "TFTModel"
]


# Lưu thông tin traceback study và error city trong quá trình optimize
l_study_city ={}
l_errCity =[]

if __name__ == '__main__':
  for nstep in range(1,args.n_predicted_period_months+1):
    print("⭐️ Nstep: ",nstep)
    lags_by_nstep = args.look_back + nstep - 1
    lags_past_covariates_by_nstep = [-lags_by_nstep+2,-lags_by_nstep+1,-lags_by_nstep] #Mảng này chứa ba giá trị tương ứng cho args.lookback 3
    lags_past_covariates_in_str = str(-lags_by_nstep+2)+","+str(-lags_by_nstep+1)+","+str(-lags_by_nstep)
    for model_name in model_name_list: 
      print("⭐️ Model_name: ",model_name)
      best_param = pd.DataFrame()
      for city_index in range(len(cities)):
        print("⭐️ City: ",cities[city_index])
        # Use Tree-structured Parzen Estimator sampler to minimise RMSE
        sampler = optuna.samplers.TPESampler()
        study = optuna.create_study(sampler=sampler, direction='minimize', study_name = model_name)
        # truyền multiple param vào trong biến trial
        obj_func = lambda trial: objective(model_name, trial, cities[city_index], nstep = nstep)
        try:
          # Optimise over 100 trials
          study.optimize(obj_func, n_trials=args.ntry, n_jobs=args.njob)

          # Print results
          print("Study statistics for : ")
          print("  Number of finished trials: ", len(study.trials))
          print("Best trial of city: ",cities[city_index])

          best_trial = study.best_trial
          # lưu best param vào trong biến toàn cục

          if model_name == "LinearRegressionModel":
            one_city_param = pd.DataFrame({
                                'City':  cities[city_index],
                                'Alg_name': 'LinearRegressionModel',
                                'Best_value': best_trial.value,
                                'n_try_opt': args.ntry,
                                'lags' : lags_by_nstep,
                                'lags_past_covariates': [lags_past_covariates_in_str],
                                'output_chunk_length': 1,
                                'random_state':best_trial.params['random_state'],
                                })
          elif model_name == "LightGBMModel":
            one_city_param = pd.DataFrame({
                                'City':  cities[city_index],
                                'Alg_name': 'LightGBMModel',
                                'Best_value': best_trial.value,
                                'n_try_opt': args.ntry,
                                'lags': lags_by_nstep,
                                'lags_past_covariates': [lags_past_covariates_in_str],
                                'multi_models': best_trial.params['multi_models'],
                                'num_leaves': best_trial.params['num_leaves'], 
                                'feature_fraction': best_trial.params['feature_fraction'], 
                                'min_child_samples': best_trial.params['min_child_samples'], 
                                'lambda_l1': best_trial.params['lambda_l1'], 
                                'lambda_l2': best_trial.params['lambda_l2'], 
                                'likelihood': best_trial.params['likelihood'], 
                                'learning_rate': best_trial.params['learning_rate']})
          elif model_name == "CatBoostModel":
            one_city_param = pd.DataFrame({
                                'City':  cities[city_index],
                                'Alg_name': 'CatBoost',
                                'Best_value': best_trial.value,
                                'n_try_opt': args.ntry,
                                'lags' : lags_by_nstep,
                                'lags_past_covariates': [lags_past_covariates_in_str],
                                'output_chunk_length': 1,
                                'likelihood': best_trial.params['likelihood'],
                                'learning_rate': best_trial.params['learning_rate'],
                                'n_estimators': best_trial.params['n_estimators'],
                                'max_depth': best_trial.params['max_depth'],
                                'bagging_temperature': best_trial.params['bagging_temperature'],
                                'l2_leaf_reg': best_trial.params['l2_leaf_reg'],
                                'random_strength':best_trial.params['random_strength'],
                                })
          elif model_name == "NHiTSModel":
            one_city_param = pd.DataFrame({
                                'City':  cities[city_index],
                                'Alg_name': 'N-HiTS',
                                'Best_value': best_trial.value,
                                'n_try_opt': args.ntry,
                                'input_chunk_length' : lags_by_nstep,
                                'output_chunk_length' : 1,
                                'MaxPool1d' : best_trial.params['MaxPool1d'],
                                'dropout' : best_trial.params['dropout'],
                                'n_epochs' : best_trial.params['n_epochs'],
                                'random_state' : best_trial.params['random_state'],
                                })
          elif model_name == "TCNModel":
            one_city_param = pd.DataFrame({
                                'City':  cities[city_index],
                                'Alg_name': 'TCNModel',
                                'Best_value': best_trial.value,
                                'n_try_opt': args.ntry,
                                'input_chunk_length': lags_by_nstep,
                                'output_chunk_length': 1,
                                'n_epochs':best_trial.params['n_epochs'],
                                'num_filters':best_trial.params['num_filters'],
                                'weight_norm':best_trial.params['weight_norm'],
                                'dilation_base':best_trial.params['dilation_base'],
                                'dropout':best_trial.params['dropout'],
                                'learning_rate':best_trial.params['learning_rate'],
                                'year':best_trial.params['year'],
                                })
          elif model_name == "NBEATSModel":
            one_city_param = pd.DataFrame({
                                'City':  cities[city_index],
                                'Alg_name': 'NBeatsModel',
                                'Best_value': best_trial.value,
                                'n_try_opt': args.ntry,
                                'output_chunk_length': 1,
                                'input_chunk_length': lags_by_nstep,
                                'n_epochs':best_trial.params['n_epochs'],
                                'dropout':best_trial.params['dropout'],
                                'random_state':best_trial.params['random_state'],
                                })  
          elif model_name == "TFTModel":
            one_city_param = pd.DataFrame({
                                'City':  cities[city_index],
                                'Alg_name': 'TFTModel',
                                'Best_value': best_trial.value,
                                'n_try_opt': args.ntry,
                                'output_chunk_length': 1,
                                'input_chunk_length': lags_by_nstep,
                                'add_relative_index': True,
                                'random_state':best_trial.params['random_state'],
                                'n_epochs':best_trial.params['n_epochs'],
                                'dropout':best_trial.params['dropout']
                                })
          elif model_name == "RandomForest":
            one_city_param = pd.DataFrame({
                                'City':  cities[city_index],
                                'Alg_name': 'RandomForest',
                                'Best_value': best_trial.value,
                                'n_try_opt': args.ntry,
                                'lags' : lags_by_nstep,
                                'lags_past_covariates': [lags_past_covariates_in_str],
                                'output_chunk_length': 1,
                                'n_estimators': best_trial.params['n_estimators'],
                                'max_depth': best_trial.params['max_depth'],
                                'random_state':best_trial.params['random_state'],
                                })
          elif model_name == "BlockRNNModel":
            one_city_param = pd.DataFrame({
                                'City':  cities[city_index],
                                'Alg_name': 'BlockRNNModel',
                                'Best_value': best_trial.value,
                                'n_try_opt': args.ntry,
                                'input_chunk_length': lags_by_nstep,
                                'output_chunk_length': 1,
                                'random_state':best_trial.params['random_state'],
                                'n_epochs':best_trial.params['n_epochs'],
                                'hidden_dim': best_trial.params['hidden_dim'],
                                'n_rnn_layers': best_trial.params['n_rnn_layers'],
                                'dropout':best_trial.params['dropout']
                                })
          file_path = 'opt_results/261023_DF_opt_hyperparam_'+ model_name + '_'+str(nstep)+'-nstep.xlsx'
          if(os.path.isfile(file_path)):
              with pd.ExcelWriter(file_path,mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer:
                  one_city_param.to_excel(writer, header=None, startrow=city_index+1,index=False)
          else:
              with pd.ExcelWriter(file_path,engine="openpyxl") as writer:
                  one_city_param.to_excel(writer, startrow=city_index,index=False)
        except:# có error thì lưu vào l_errCity để check lại sau 
          l_errCity.append(cities[city_index])

[I 2023-10-26 18:49:45,399] A new study created in memory with name: RandomForest


⭐️ Nstep:  1
⭐️ Model_name:  RandomForest
⭐️ City:  An Giang


[I 2023-10-26 18:49:45,661] Trial 0 finished with value: 5.07890073725021 and parameters: {'random_state': 23, 'n_estimators': 157, 'max_depth': 6}. Best is trial 0 with value: 5.07890073725021.
[I 2023-10-26 18:49:45,670] A new study created in memory with name: RandomForest
[I 2023-10-26 18:49:45,763] Trial 0 finished with value: 12.389449892813808 and parameters: {'random_state': 28, 'n_estimators': 66, 'max_depth': 15}. Best is trial 0 with value: 12.389449892813808.
[I 2023-10-26 18:49:45,771] A new study created in memory with name: RandomForest


mean_squared_error: 74.1253
rmse: 8.609607319135275
mape: 55854.90580350234
Study statistics for : 
  Number of finished trials:  1
Best trial of city:  An Giang
⭐️ City:  BR Vũng Tàu
mean_squared_error: 328.0576
rmse: 18.112359381844534
mape: 10.844015449123416
Study statistics for : 
  Number of finished trials:  1
Best trial of city:  BR Vũng Tàu
⭐️ City:  Bình Phước


[I 2023-10-26 18:49:45,991] Trial 0 finished with value: 11.975873609804713 and parameters: {'random_state': 39, 'n_estimators': 185, 'max_depth': 11}. Best is trial 0 with value: 11.975873609804713.
[I 2023-10-26 18:49:45,997] A new study created in memory with name: RandomForest
[I 2023-10-26 18:49:46,108] Trial 0 finished with value: 3.048662701248804 and parameters: {'random_state': 10, 'n_estimators': 98, 'max_depth': 5}. Best is trial 0 with value: 3.048662701248804.
[I 2023-10-26 18:49:46,115] A new study created in memory with name: RandomForest


...

⭐️ City:  Bình Phước
⭐️ City:  Bình Thuận
⭐️ City:  Bình Định
⭐️ Model_name:  NHiTSModel
⭐️ City:  An Giang
⭐️ City:  BR Vũng Tàu
⭐️ City:  Bình Phước
⭐️ City:  Bình Thuận
⭐️ City:  Bình Định


[W 2023-10-26 18:51:38,485] Trial 0 failed with parameters: {'random_state': 39, 'dropout': 0.42111891975195903, 'n_epochs': 160, 'MaxPool1d': True} because of the following error: TypeError("__init__() got an unexpected keyword argument 'auto_select_gpus'").
Traceback (most recent call last):
  File "/Users/trinhtruc/Library/Python/3.9/lib/python/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/r_/lbw3rw192wl9sx9vtc1c2_xc0000gn/T/ipykernel_36494/528832268.py", line 37, in <lambda>
    obj_func = lambda trial: objective(model_name, trial, cities[city_index], nstep = nstep)
  File "/var/folders/r_/lbw3rw192wl9sx9vtc1c2_xc0000gn/T/ipykernel_36494/1386761.py", line 208, in objective
    mae_error = output_prediction_for_location(df_train, df_valid, model, location=city, feature_list=selected_features,
  File "/var/folders/r_/lbw3rw192wl9sx9vtc1c2_xc0000gn/T/ipykernel_36494/1283319320.py", line 12, in output_prediction

⭐️ Model_name:  TCNModel
⭐️ City:  An Giang
⭐️ City:  BR Vũng Tàu
⭐️ City:  Bình Phước
⭐️ City:  Bình Thuận
⭐️ City:  Bình Định
⭐️ Model_name:  TFTModel
⭐️ City:  An Giang
⭐️ City:  BR Vũng Tàu
⭐️ City:  Bình Phước


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[W 2023-10-26 18:51:38,699] Trial 0 failed with parameters: {'random_state': 42, 'dropout': 0.5892361143456858, 'n_epochs': 159} because of the following error: TypeError("Cannot convert a MPS Tensor to float64 dtype as the MPS framework doesn't support float64. Please use float32 instead.").
Traceback (most recent call last):
  File "/Users/trinhtruc/Library/Python/3.9/lib/python/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/r_/lbw3rw192wl9sx9vtc1c2_xc0000gn/T/ipykernel_36494/528832268.py", line 37, in <lambda>
    obj_func = lambda trial: objective(model_name, trial, cities[city_index], nstep = nstep)
  File "/var/folders/r_/lbw3rw192wl9sx9vtc1c2_xc0000gn/T/ipykernel_36494/1386761.py", line 208, in objective
    mae_error = output_prediction_for_location(df_trai

⭐️ City:  Bình Thuận
⭐️ City:  Bình Định


In [407]:
send_to_telegram("Chạy xong optimize rùiii!!Vô check thuiii!!!" )

{"ok":true,"result":{"message_id":1275,"sender_chat":{"id":-1001712314864,"title":"PTN_Announcement","username":"ptn_announcement","type":"channel"},"chat":{"id":-1001712314864,"title":"PTN_Announcement","username":"ptn_announcement","type":"channel"},"date":1698320835,"text":"Ch\u1ea1y xong optimize r\u00f9iii!!V\u00f4 check thuiii!!!"}}
